In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
os.chdir("data/shots")

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
print(all_filenames)

(The cell below is to be executed only once. Number of rows has to be 210072)

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv("/Users/oksanahrytsiv/Documents/GitHub/NBA_BigDataProject/data/combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
df = pd.DataFrame(combined_csv)

In [ ]:
df.shape

In [ ]:
df = df.drop(['self previous shot', 'opponent previous shot', 'time from last shot'], axis = 1)
df = df.rename(columns={'player position':'player_position','shot type':'shot_type','away team': 'away_team','current shot outcome':'outcome', 'home team' : 'home_team', 'location x':'loc_x','location y':'loc_y','shoot player':'shooter','home game':'home'})

In [ ]:
df.head()

In [ ]:
df_home = df[df.home=='Yes']

In [ ]:
df_away = df[df.home=='No']

In [ ]:
mirror_q_away = [1,2]
mirror_q_home = [3,4,5,6,7,8]

In [ ]:
middle_x = 470.0
middle_y = 250.0

In [ ]:
df_home_mirror = df_home[df_home.quarter.isin(mirror_q_home)]
df_home_notmirror = df_home[~df_home.quarter.isin(mirror_q_home)]
df_away_mirror = df_away[df_away.quarter.isin(mirror_q_away)]
df_away_notmirror = df_away[~df_away.quarter.isin(mirror_q_away)]

In [ ]:
df_home.shape, df_home_mirror.shape, df_home_notmirror.shape, df_away.shape, df_away_mirror.shape, df_away_notmirror.shape

In [ ]:
df_home_mirror = df_home_mirror[df_home_mirror.loc_x >= middle_x]
df_home_notmirror = df_home_notmirror[df_home_notmirror.loc_x <= middle_x]
df_away_mirror = df_away_mirror[df_away_mirror.loc_x >= middle_x]
df_away_notmirror = df_away_notmirror[df_away_notmirror.loc_x <= middle_x]

In [ ]:
df_home.shape, df_home_mirror.shape, df_home_notmirror.shape, df_away.shape, df_away_mirror.shape, df_away_notmirror.shape

In [ ]:
df_home_mirror['loc_x'] = df_home_mirror['loc_x'].apply(lambda row: row - 2*(row-middle_x))
df_home_mirror['loc_y'] = df_home_mirror['loc_y'].apply(lambda row: row - 2*(row-middle_y) if row > 250.0 else row + 2*(middle_y-row))

In [ ]:
df_away_mirror['loc_x'] = df_away_mirror['loc_x'].apply(lambda row: row - 2*(row-middle_x))
df_away_mirror['loc_y'] = df_away_mirror['loc_y'].apply(lambda row: row - 2*(row-middle_y) if row > 250.0 else row + 2*(middle_y-row))

In [ ]:
df_home_mirror.shape

In [ ]:
df_home = df_home_notmirror.append(df_home_mirror)
df_away = df_away_notmirror.append(df_away_mirror)

In [ ]:
df_home.shape, df_away.shape

In [ ]:
df = df_home.append(df_away)

In [ ]:
df.shape

In [ ]:
from courts import Rectangle, Circle, Arc

def draw_half_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()
     
    backboard1 = Rectangle((50, 225), -1, 50, linewidth=lw, color=color)
    
    hoop1 = Circle((55, 250), radius=6, linewidth=lw, color=color, fill=False)
    
    restricted1 = Arc((50, 250), 80, 80, angle=270, theta1=0, theta2=180, linewidth=lw,
                     color=color)
    
    freethrow1_outer = Arc((195, 250), 100, 120, angle=270, theta1=0, theta2=180, linewidth=lw,
                     color=color)
    
    freethrow1_inner = Arc((195, 250), 100, 120, angle=270, theta1=180, theta2=0, linewidth=lw,
                     color=color, linestyle='dashed')
    
    freethrow1_point = Circle((195, 250), radius=4, linewidth=lw, color=color, fill=True)
    
    threepoint1 = Arc((90, 250), 450, 400, angle=270, theta1=0, theta2=180, linewidth=lw,
                     color=color)
    
    innerbox1 = Rectangle((20, 200), 175, 100, linewidth=lw, color=color,
                          fill=False)
    
    outbox1 = Rectangle((20, 190), 175, 120, linewidth=lw, color=color,
                      fill=False)
    
    half_court_circle  = Arc((490, 250), 100, 120, angle=270, theta1=180, theta2=0, linewidth=lw,
                     color=color)
    
    half_court_point  = Arc((490, 250), 5, 5, angle=270, theta1=180, theta2=0, linewidth=6,
                 color=color)
    
    lower_three_line = Rectangle((20, 25), 70, 1, linewidth=0.25*lw, color=color,
                          fill=True)
    
    upper_three_line = Rectangle((20, 475), 70, 1, linewidth=0.25*lw, color=color,
                      fill=True)
    
    # List of the court elements to be plotted onto the axes
    court_elements = [backboard1, hoop1, restricted1,
                       threepoint1, innerbox1, freethrow1_outer,freethrow1_point,
                      outbox1, freethrow1_inner,lower_three_line, upper_three_line, half_court_circle, half_court_point]
    
    if outer_lines:
    # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((20, 0), 470, 500, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)
    
      # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)
    
    ax.set_aspect('auto')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(labelbottom=True, labelleft=True)
#     ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3, fancybox=True, shadow=True)
    
    return ax

In [ ]:
df.head(2)

In [ ]:
plt.figure(figsize=(8,10))
draw_half_court(outer_lines=True)
sns.scatterplot(x='loc_x',y='loc_y',hue='outcome',data=df)
plt.xlim(0,510)
plt.ylim(-20,520)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.show()

In [ ]:
#export_csv = df.to_csv('shots_dataframe.csv', index = None, header=False)